<a href="https://colab.research.google.com/github/RhysWangJunfei/nilm/blob/master/nilm_binary.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import io
from google.colab import files
import pandas as pd
import tensorflow as tf
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split 
from sklearn.ensemble import GradientBoostingClassifier as gbdt

'''Sliding window function'''
def create_dataset(dataset, look_back=1):
    dataX = []
    for i in range(len(dataset)-look_back+1):
        a = dataset[i:(i+look_back)]
        dataX.append(a)
    return np.array(dataX)

uploaded = files.upload()
'''Load data'''
WHE_data = pd.read_csv(io.BytesIO(uploaded['Electricity_WHE.csv']))[['unix_ts','P']]
uploaded = files.upload()
CWE_data = pd.read_csv(io.BytesIO(uploaded['Electricity_CWE.csv']))['P']

WHE_data['datetime'] = pd.to_datetime(WHE_data['unix_ts'],unit='s')
WHE_data = WHE_data.drop(['unix_ts'],axis=1)

WHE_data['month'] = WHE_data['datetime'].dt.month
WHE_data['hour'] = WHE_data['datetime'].dt.hour
WHE_data['weekday'] = WHE_data['datetime'].dt.dayofweek
WHE_data['quarter'] = WHE_data['datetime'].dt.quarter

Saving Electricity_WHE.csv to Electricity_WHE.csv


Saving Electricity_CWE.csv to Electricity_CWE.csv


In [0]:
window_size=10
'''Using sliding window to build training examples'''
time_data = create_dataset(WHE_data['P'].as_matrix(), window_size)

data_month = WHE_data['month'].as_matrix()[window_size-1:].reshape([-1,1])
data_hour = WHE_data['hour'].as_matrix()[window_size-1:].reshape([-1,1])
data_weekday = WHE_data['weekday'].as_matrix()[window_size-1:].reshape([-1,1])
dataX = np.hstack([time_data,data_month,data_hour,data_weekday])

print(1)
#get y for the single load
cwe_Y = CWE_data[window_size-1:].values.reshape([CWE_data.shape[0]-window_size+1,1])
cweY=cwe_Y[cwe_Y>0].reshape(-1,1)
cweX=dataX[[np.where(cwe_Y>0)[0]],:][0]

X_train, X_test, y_train, y_test = train_test_split(dataX, cwe_Y, test_size=0.2, shuffle=True)
scaler = MinMaxScaler()
scaler.fit(X_train)
X_test = scaler.transform(X_test)


1


/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:595: DataConversionWarning: Data with input dtype int64 was converted to float64 by MinMaxScaler.
  warnings.warn(msg, DataConversionWarning)


In [0]:
plant
from sklearn.ensemble import GradientBoostingClassifier as gbdt
cls = gbdt()
cls.fit(X_train,y_train)
y_pre = cls.pre(X_test)

NameError: ignored

In [0]:
acc = y_pre*y_test
print(acc)

NameError: ignored